naive: comes from the assumption that the features in a dataset are mutually independent.

conditional probability الترتيب مهم

N-Grams

stemming جذر الكلمة and lemmatization الكلمة في المعجم

Text generation (chat bot, translation, predict next word, summarization) – text classification – grammar checker – autocorrect – answering questions like google

In [25]:
import pandas as pd
import numpy as np

data = pd.read_csv('data.csv')
data.head()

,text,class
0,Klement Gottwaldi surnukeha palsameeriti ning ...,est\n
1,"Sebes, Joseph; Pereira Thomas (1961) (på eng)....",swe\n
2,भारतीय स्वातन्त्र्य आन्दोलन राष्ट्रीय एवम क्षे...,mai\n
3,"Après lo cort periòde d'establiment a Basilèa,...",oci\n
4,ถนนเจริญกรุง (อักษรโรมัน: Thanon Charoen Krung...,tha\n


In [26]:
label = pd.read_csv('labels.csv', delimiter=';')
label.head()

,Label,English,Wiki Code,ISO 369-3,German,Language family,Writing system,Remarks,Synonyms
0,ace,Achinese,ace,ace,Achinesisch,Austronesian,NaN,NaN,NaN
1,afr,Afrikaans,af,afr,Afrikaans,Indo-European,NaN,NaN,NaN
2,als,Alemannic German,als,gsw,Alemannisch,Indo-European,NaN,(ursprünglich nur Elsässisch),NaN
3,amh,Amharic,am,amh,Amharisch,Afro-Asiatic,NaN,NaN,NaN
4,ang,Old English,ang,ang,Altenglisch,Indo-European,NaN,(ca. 450-1100),Angelsächsisch


In [27]:
data['class']=data['class'].apply(lambda x:x.replace('\n',''))

In [28]:
data.head()

,text,class
0,Klement Gottwaldi surnukeha palsameeriti ning ...,est
1,"Sebes, Joseph; Pereira Thomas (1961) (på eng)....",swe
2,भारतीय स्वातन्त्र्य आन्दोलन राष्ट्रीय एवम क्षे...,mai
3,"Après lo cort periòde d'establiment a Basilèa,...",oci
4,ถนนเจริญกรุง (อักษรโรมัน: Thanon Charoen Krung...,tha


In [29]:
lang = ['eng','ita','fra','spa','deu']
reduced_data = data[data['class'].isin(lang)]

In [30]:
reduced_data['class'].value_counts()

fra    1000
spa    1000
ita    1000
eng    1000
deu    1000
Name: class, dtype: int64

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
d = count_vectorizer.fit_transform(reduced_data['text'])
d

<5000x57882 sparse matrix of type '<class 'numpy.int64'>'
	with 243855 stored elements in Compressed Sparse Row format>

In [33]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(d.toarray(), reduced_data['class'], test_size=0.2, random_state=42)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [34]:
accuracy = np.mean(y_pred == y_test)
accuracy

0.99

In [35]:
from sklearn.pipeline import Pipeline
#from mlxtend.preprocessing import DenseTransformer 
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.naive_bayes import GaussianNB
class ToArray():
    def fit(self, X, y, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.toarray()
        
text_clf = Pipeline([ 
    ('vect', CountVectorizer()), 
    ('toArray', ToArray()),
    ('clf', GaussianNB())
    ]) 


In [36]:
text_clf.fit(reduced_data['text'], reduced_data['class'])

Pipeline(steps=[('vect', CountVectorizer()),
                ('toArray', <__main__.ToArray object at 0x000002528D6D40D0>),
                ('clf', GaussianNB())])

In [37]:
text_clf.predict(['Je suis une fille'])

array(['fra'], dtype='<U3')

In [43]:
import ipywidgets as widgets
widgets.Text(value='Enter your sentence here.')


Text(value='Enter your sentence here.')

In [56]:
def identify_language(text):
    return text_clf.predict([text])[0]

In [57]:
widgets.interact(identify_language, text="Enter your sentence here.");

interactive(children=(Text(value='Enter your sentence here.', description='text'), Output()), _dom_classes=('w…